## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: K562
# EMBRACENET

da fare 
- sistema selezione augmentation in embracenet
- crea grafici AUPRC reti

In [1]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# create a database to store optuna studies with sqlite backend
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [3]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [4]:
cell_line = CELL_LINES[5]
cell_line

'K562'

---

In [5]:
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline
from BIOINF_tesi.models import EmbraceNetMultimodal, ConcatNetMultimodal
from BIOINF_tesi.models.utils import Kfold_CV_Multimodal

In [6]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [41]:
task = TASKS[0]

In [8]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [7]:
model=EmbraceNetMultimodal

In [10]:
# IMBALANCED
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:33,233] Using an existing study with name 'K562_active_E_vs_inactive_E_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 


/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.08764519665835224
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 512
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 1.0997980747877431e-05
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.6933956426792006
    weight_decay: 0.0013492109869972988

===============> MODEL TESTING
AUPRC test score: 0.14303254167212115


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:34,262] Using an existing study with name 'K562_active_E_vs_inactive_E_EmbraceNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.0906023981027689
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 512
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.2
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 64
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 64
    lr: 0.0037425533109937707
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.4812967632292007
    weight_decay: 0.0010571041689318545

===

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:34,734] Using an existing study with name 'K562_active_E_vs_inactive_E_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


AUPRC test score: 0.0867229394240318


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.0925009619084958
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 64
    CNN_out_channels_l3: 256
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 128
    lr: 1.1313730164017063e-05
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.476205399441

In [11]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [10]:
model=ConcatNetMultimodal

In [28]:
# IMBALANCED
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:34:48,109] Using an existing study with name 'K562_active_E_vs_inactive_E_ConcatNetMultimodal_1' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.15109354238996053
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CONCATNET_dropout_l0: 0.2
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 1024
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 64
    lr: 0.00719945182925327
    optimizer: Adam
    weight_decay: 0.00011469333801704875

===============> MODEL TESTING
AUPRC test score: 0.0864799126862657


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:34:49,732] Using an existing study with name 'K562_active_E_vs_inactive_E_ConcatNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.16206917354907238
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.2
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 512
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.0009999975743514892
    optimizer: Nadam
    weight_decay: 0.0001835707608361668

===============> MODEL TESTING
AUPRC test score: 0.1096184021298494


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:34:50,333] Using an existing study with name 'K562_active_E_vs_inactive_E_ConcatNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1637127192015848
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 128
    CONCATNET_dropout_l0: 0.3
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 768
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 2.8367509823282757e-05
    optimizer: Adam
    weight_decay: 0.0001586432038529753

===============> MODEL TESTING
AUPRC test score: 0.14820288102510312



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.11477


In [29]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [30]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [42]:
model=EmbraceNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (49660) in class 0 will be larger than the number of samples in the majority class (class #0 -> 25643)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 08:56:25,536] Using an existing study with name 'K562_active_E_vs_inactive_E_EmbraceNetMultimodal_1augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.08763204225352113
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 256
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.02370337523388928
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.38593923981832934
    weight_decay: 0.020092471466273788

===============> MODEL TESTING
AUPRC test score: 0.08647467725918571


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (48680) in class 0 will be larger than the number of samples in the majority class (class #0 -> 25692)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 08:57:22,027] Using an existing study with name 'K562_active_E_vs_inactive_E_EmbraceNetMultimodal_2augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (73820) in class 0 will be larger than the number of samples in the majority class (clas

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.16635749078720682
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 256
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    lr: 0.0009603471607112739
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.6125658520780805
    weight_decay: 0.00010006306435679932

===============> MODEL TESTING
AUPRC test score: 0.1545400719214726


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (49300) in class 0 will be larger than the number of samples in the majority class (class #0 -> 25661)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 08:58:16,303] Using an existing study with name 'K562_active_E_vs_inactive_E_EmbraceNetMultimodal_3augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (73060) in class 0 will be larger than the number of samples in the majority class (clas

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1766841667928493
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 64
    lr: 0.000575420828618642
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.36515803553177695
    weight_decay: 0.00013016232467942407

===============> MODEL TESTING
AUPRC test score: 0.17047699272685407



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.13716


In [44]:
from BIOINF_tesi.visual import parse_output_for_params_dict
parse_output_for_params_dict(o, cell_line, task, model, device, verbose=True, augmentation=True)

0


OrderedDict([('CNN_dropout_l0', 0.4),
             ('CNN_dropout_l1', 0.0),
             ('CNN_kernel_size_l0', 15.0),
             ('CNN_kernel_size_l1', 15.0),
             ('CNN_n_layers', 2.0),
             ('CNN_out_channels_l0', 64.0),
             ('CNN_out_channels_l1', 96.0),
             ('EMBRACENET_dropout_l0', 0.2),
             ('EMBRACENET_embracement_size', 1024.0),
             ('EMBRACENET_n_units_l0', 256.0),
             ('FFNN_dropout_l0', 0.3),
             ('FFNN_n_layers', 1.0),
             ('FFNN_n_units_l0', 32.0),
             ('lr', 0.02370337523388928),
             ('n_post_layers', 1.0),
             ('optimizer', 'Adam'),
             ('selection_probabilities_FFNN', 0.38593923981832934),
             ('weight_decay', 0.020092471466273788)])

1


OrderedDict([('CNN_dropout_l0', 0.0),
             ('CNN_dropout_l1', 0.5),
             ('CNN_dropout_l2', 0.4),
             ('CNN_dropout_l3', 0.4),
             ('CNN_kernel_size_l0', 5.0),
             ('CNN_kernel_size_l1', 11.0),
             ('CNN_kernel_size_l2', 11.0),
             ('CNN_kernel_size_l3', 11.0),
             ('CNN_n_layers', 4.0),
             ('CNN_out_channels_l0', 64.0),
             ('CNN_out_channels_l1', 96.0),
             ('CNN_out_channels_l2', 128.0),
             ('CNN_out_channels_l3', 256.0),
             ('EMBRACENET_embracement_size', 768.0),
             ('FFNN_dropout_l0', 0.2),
             ('FFNN_dropout_l1', 0.4),
             ('FFNN_n_layers', 2.0),
             ('FFNN_n_units_l0', 128.0),
             ('FFNN_n_units_l1', 16.0),
             ('lr', 0.0009603471607112739),
             ('n_post_layers', 0.0),
             ('optimizer', 'Nadam'),
             ('selection_probabilities_FFNN', 0.6125658520780805),
             ('weight_decay',

2


OrderedDict([('CNN_dropout_l0', 0.2),
             ('CNN_kernel_size_l0', 5.0),
             ('CNN_n_layers', 1.0),
             ('CNN_out_channels_l0', 32.0),
             ('EMBRACENET_embracement_size', 1024.0),
             ('FFNN_dropout_l0', 0.3),
             ('FFNN_dropout_l1', 0.0),
             ('FFNN_n_layers', 2.0),
             ('FFNN_n_units_l0', 32.0),
             ('FFNN_n_units_l1', 64.0),
             ('lr', 0.000575420828618642),
             ('n_post_layers', 0.0),
             ('optimizer', 'Nadam'),
             ('selection_probabilities_FFNN', 0.36515803553177695),
             ('weight_decay', 0.00013016232467942407)])

In [11]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [13]:
task = TASKS[1]
task

'active_P_vs_inactive_P'

In [14]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [14]:
model=EmbraceNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:40,096] Using an existing study with name 'K562_active_P_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.31486753382739113
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 96
    CNN_out_channels_l3: 128
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 128
    lr: 0.00984353705845039
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.8182449098603893
    weight_decay: 0.01003426405550768

===============> MODEL TESTING
AUPRC test score: 0.32009363922688366


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:40,657] Using an existing study with name 'K562_active_P_vs_inactive_P_EmbraceNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3443267908822439
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 256
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.0011718727412020741
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.4807835648207964
    weight_decay: 0.0001251709429610383

===============> MODEL TESTING
AUPRC test score: 0.3443778647649518


>>> ITERATION N. 3


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:41,334] Using an existing study with name 'K562_active_P_vs_inactive_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.34450370493014115
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 96
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.0007896513211772321
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.5319490687604308
    weight_decay: 0.00023676465649062538

===============> MODEL TESTING
AUPRC test score: 0.3431341667809268



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.33587


In [17]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [17]:
model=ConcatNetMultimodal

In [34]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:34:57,911] Using an existing study with name 'K562_active_P_vs_inactive_P_ConcatNetMultimodal_1' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3051213369813639
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 128
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.2
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 32
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    lr: 3.921667749800381e-05
    optimizer: Adam
    weight_decay: 0.00016726424483515291

===============> MODEL TESTING
AUPRC test score: 0.35331882719672225


>>> ITERATION N. 2


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:34:58,813] Using an existing study with name 'K562_active_P_vs_inactive_P_ConcatNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3378047353449225
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 96
    CNN_out_channels_l3: 128
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.2
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 64
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 64
    lr: 0.004813608976253827
    optimizer: Nadam
    weight_decay: 0.00010305280052524885

===============> MODEL TESTING
AUPRC test score: 0.3474940368450853


>>> ITERATION N. 3

==========

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:34:59,415] Using an existing study with name 'K562_active_P_vs_inactive_P_ConcatNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.24264707206227515
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 256
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.3
    CONCATNET_dropout_l2: 0.3
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 512
    CONCATNET_n_units_l2: 16
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 16
    lr: 0.00042651458814273196
    optimizer: Nadam
    weight_decay: 0.0002709769190115885

===============> MODEL TESTING
AUPRC test score: 0.2351014681176871



3-FOLD CROSS-VALIDATION AUPRC TEST S

In [35]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [36]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [15]:
model=EmbraceNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (77464) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38732)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 08:59:32,201] Using an existing study with name 'K562_active_P_vs_inactive_P_EmbraceNetMultimodal_1augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (116516) in class 0 will be larger than the number of samples in the majority class (cla

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3474238128572922
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.3
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 128
    lr: 0.0002856571404222289
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.910430823944452
    weight_decay: 0.0002796787837665966

===============> MODEL TESTING
AUPRC test score: 0.3648196903878952


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (77638) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38819)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:01:30,346] Using an existing study with name 'K562_active_P_vs_inactive_P_EmbraceNetMultimodal_2augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (116364) in class 0 will be larger than the number of samples in the majority class (cla

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.20996173068327084
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_dropout_l1: 0.0
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 512
    EMBRACENET_n_units_l1: 16
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 4
    FFNN_n_units_l3: 16
    lr: 6.406190767687052e-05
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.14470139906231827
    weight_decay: 0.002194312615192964

===============> MODEL TESTING
AUPRC test score: 0.23188809811440023


>>> ITERATION N. 3

===============> HYPERPAR

/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (77722) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38861)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:03:31,690] Using an existing study with name 'K562_active_P_vs_inactive_P_EmbraceNetMultimodal_3augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (116548) in class 0 will be larger than the number of samples in the majority class (cla

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2873060747534889
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 32
    lr: 0.01443384284991165
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.9084480196043856
    weight_decay: 0.0034664700841699238

===============> MODEL TESTING
AUPRC test score: 0.29678801088050794



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.29783


In [17]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [20]:
task = TASKS[2]
task

'active_E_vs_active_P'

In [20]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [21]:
model=EmbraceNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:45,788] Using an existing study with name 'K562_active_E_vs_active_P_EmbraceNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7450591113498456
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 128
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 16
    lr: 3.3151363709364785e-05
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.7536428600508083
    weight_decay: 0.0001661726226346702

===============> MODEL TESTING
AUPRC test score: 0.7814256533916237




/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:46,005] Using an existing study with name 'K562_active_E_vs_active_P_EmbraceNetMultimodal_1_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7237891914105221
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 256
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 64
    EMBRACENET_n_units_l1: 32
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 32
    lr: 1.6714676359934622e-05
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.6377778714085374
    weight_decay

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 08:44:46,256] Using an existing study with name 'K562_active_E_vs_active_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


AUPRC test score: 0.759198813750173


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.750362940420709
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 64
    lr: 7.668884120471917e-05
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.1865827557130616
    weight_decay: 0.0001674356974238721

===============> MODEL TESTING
AUPRC test score: 0.7523892025906109



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.76434


In [23]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [24]:
model=ConcatNetMultimodal

In [40]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:35:05,743] Using an existing study with name 'K562_active_E_vs_active_P_ConcatNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7524088554501104
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.5
    CONCATNET_dropout_l2: 0.3
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 32
    CONCATNET_n_units_l2: 128
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 64
    lr: 0.003108113345707817
    optimizer: Nadam
    weight_decay: 0.0004442933217076603


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:35:06,014] Using an existing study with name 'K562_active_E_vs_active_P_ConcatNetMultimodal_1_2' instead of creating a new one.



===============> MODEL TESTING
AUPRC test score: 0.7371535965248306


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 21:35:06,229] Using an existing study with name 'K562_active_E_vs_active_P_ConcatNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7297132863678131
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 512
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.3
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 128
    CONCATNET_n_units_l2: 16
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.005051846096718053
    optimizer: Nadam
    weight_decay: 0.010612009090492307

===============> MODEL TESTING
AUPRC test score: 0.3927974538098884


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
S

In [41]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [42]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [21]:
model=EmbraceNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (11172) in class 0 will be larger than the number of samples in the majority class (class #0 -> 5586)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:04:55,225] Using an existing study with name 'K562_active_E_vs_active_P_EmbraceNetMultimodal_1augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (16756) in class 0 will be larger than the number of samples in the majority class (class #

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7596111881299012
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.3
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 128
    lr: 0.014371973604253983
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.6347299274462684
    weight_decay: 0.0016135563737140258

===============> MODEL TESTING
AUPRC test score: 0.78604604098981


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (11140) in class 0 will be larger than the number of samples in the majority class (class #0 -> 5570)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:04:58,414] Using an existing study with name 'K562_active_E_vs_active_P_EmbraceNetMultimodal_2augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (16704) in class 0 will be larger than the number of samples in the majority class (class #

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7959826750444793
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.009567585671154526
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.5539246421995385
    weight_decay: 0.00019935324628404878

===============> MODEL TESTING
AUPRC test score: 0.7945168411370399


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (11092) in class 0 will be larger than the number of samples in the majority class (class #0 -> 5546)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:05:01,664] Using an existing study with name 'K562_active_E_vs_active_P_EmbraceNetMultimodal_3augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (16636) in class 0 will be larger than the number of samples in the majority class (class #

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7484603244651804
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 5.7567319891193354e-05
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.21623917654179048
    weight_decay: 0.0013132958842830297

===============> MODEL TESTING
AUPRC test score: 0.7742494425485762



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.78494


In [23]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [27]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [8]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [28]:
model=EmbraceNetMultimodal

In [11]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 11:34:43,868] Using an existing study with name 'K562_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.
/opt/conda/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  6
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5713808325271222
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 32
    EMBRACENET_n_units_l1: 64
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.00021224021905621443
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.33064018066605183
    weight_decay: 0.0023943792991417095

===============> MODEL TESTING
AUPRC test score: 0.5740933684825239


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 11:34:47,163] Using an existing study with name 'K562_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5773265498781514
  Params: 
    CNN_dropout_l0: 0.4
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.007755532129721151
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.4657316436810155
    weight_decay: 0.00011339493658220607

===============> MODEL TESTING
AUPRC test score: 0.5798172878688741


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 11:34:48,664] Using an existing study with name 'K562_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5775551823114605
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 64
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.2
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 128
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 64
    lr: 4.426510674291985e-05
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.014330109000844082
    weight_decay: 0.007072449024155528

===============> MODEL TESTING
AUPRC test score: 0.5674128011920179



3-FOLD CROSS-VA

In [12]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [13]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [31]:
model=ConcatNetMultimodal

In [11]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-29 13:33:11,405] Using an existing study with name 'K562_inactive_E_vs_inactive_P_ConcatNetMultimodal_1' instead of creating a new one.
/opt/conda/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5774639912762131
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.0
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 1024
    CONCATNET_n_units_l1: 256
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 4
    FFNN_n_units_l3: 4
    lr: 0.00029929581576531916
    optimizer: RMSprop
    weight_decay: 0.0006906962554088203

===============> MODEL TESTING
AUPRC test score: 0.5862598780762344


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-29 13:33:15,946] Using an existing study with name 'K562_inactive_E_vs_inactive_P_ConcatNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5645793329401175
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.5
    CONCATNET_dropout_l2: 0.5
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 32
    CONCATNET_n_units_l2: 128
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 64
    lr: 0.0013206954605885933
    optimizer: RMSprop
    weight_decay: 0.00018812189052349663

===============> MODEL TESTING
AUPRC test score: 0.5938258885564006


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-29 13:33:17,805] Using an existing study with name 'K562_inactive_E_vs_inactive_P_ConcatNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.576905819275232
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.2
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 512
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 64
    lr: 0.00015756281125133535
    optimizer: Adam
    weight_decay: 0.003795772298227342

===============> MODEL TESTING
AUPRC test score: 0.5944901317797598



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.59153


In [12]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [13]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [28]:
model=EmbraceNetMultimodal

In [29]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (77532) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38766)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (51460) in class 1 will be larger than the number of samples in the majority class (class #0 -> 38766)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:49:00,733] Using an exist

Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5494686927339494
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 32
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.0
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 32
    lr: 0.001718739222379226
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.9263028037510795
    weight_decay: 0.001537982761904299

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (116404) in class 0 will be larger than the number of samples in the majority class (class #0 -> 58202)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (77086) in class 1 will be larger than the number of samples in the majority class (class #0 -> 58202)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.5382180755213448


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


...

AUPRC test score: 0.5454326731763245


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (77740) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38870)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (51254) in class 1 will be larger than the number of samples in the majority class (class #0 -> 38870)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:55:03,261] Using an exist

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5478099703461285
  Params: 
    CNN_dropout_l0: 0.4
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.0008636102636781816
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.27763728968058743
    weight_decay: 0.08942523656371981

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (116684) in class 0 will be larger than the number of samples in the majority class (class #0 -> 58342)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (76808) in class 1 will be larger than the number of samples in the majority class (class #0 -> 58342)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.5288522234180226



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.5375


In [30]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [31]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [34]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [10]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [35]:
model=EmbraceNetMultimodal

In [17]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 11:35:01,095] Using an existing study with name 'K562_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.24938576388057496
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.0001422481393253997
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.1946817498663932
    weight_decay: 0.00017441254165299075

===============> MODEL TESTING
AUPRC test score: 0.24346081055146418


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 11:35:02,909] Using an existing study with name 'K562_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.18248439990189416
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 64
    EMBRACENET_n_units_l1: 16
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.00059766732643232
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.5851677774502889
    weight_decay: 0.02474055469519636

===============> MODEL TESTING
AUPRC test score: 0.1114919856831621


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-27 11:35:04,390] Using an existing study with name 'K562_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1898264608362513
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 64
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 512
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 16
    FFNN_n_units_l3: 16
    lr: 0.0010038638892625694
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.6570191370647606
    weight_decay: 0.009107812335937214

===============> MODEL TESTING
AUPRC test score: 0.17915048575924053



3-FOLD CROSS-VALIDATION 

In [18]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [21]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [38]:
model=ConcatNetMultimodal

In [17]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-29 13:33:23,388] Using an existing study with name 'K562_active_EP_vs_inactive_rest_ConcatNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2393370800451469
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CONCATNET_dropout_l0: 0.0
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 1024
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 32
    lr: 0.001726915338217551
    optimizer: Nadam
    weight_decay: 0.0008407051462751323

===============> MODEL TESTING
AUPRC test score: 0.2600491193265578


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-29 13:33:25,290] Using an existing study with name 'K562_active_EP_vs_inactive_rest_ConcatNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.17693527016147664
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 128
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.3
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 512
    CONCATNET_n_units_l2: 64
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 128
    lr: 9.914499311762714e-05
    optimizer: RMSprop
    weight_decay: 0.012718032749082665

===============> MODEL TESTING
AUPRC test score: 0.1894883733727993


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-29 13:33:26,717] Using an existing study with name 'K562_active_EP_vs_inactive_rest_ConcatNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.23854489773064705
  Params: 
    CNN_dropout_l0: 0
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    CONCATNET_dropout_l0: 0.0
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 512
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 32
    lr: 1.1109545035969873e-05
    optimizer: Adam
    weight_decay: 0.0006327857965804624

===============> MODEL TESTING
AUPRC test score: 0.23936896420110162



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.22964


In [18]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [19]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [34]:
model=EmbraceNetMultimodal

In [35]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (129040) in class 0 will be larger than the number of samples in the majority class (class #0 -> 64520)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 09:58:52,576] Using an existing study with name 'K562_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_augmentation' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.26388182080291905
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    lr: 0.00019616016359155972
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.6699046858833698
    weight_decay: 0.004267458395085609

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (193572) in class 0 will be larger than the number of samples in the majority class (class #0 -> 96786)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.2710580139343812


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


...

AUPRC test score: 0.11249257084771282


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (129058) in class 0 will be larger than the number of samples in the majority class (class #0 -> 64529)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-02 10:09:41,329] Using an existing study with name 'K562_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_augmentation_2_augmentation_3_augmentation' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2747032521963336
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 96
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.00203577137579697
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.9077511349448815
    weight_decay: 0.0005311680685803074

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (193314) in class 0 will be larger than the number of samples in the majority class (class #0 -> 96657)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.28070730826812634



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.22142


In [36]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [37]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---